# Cut Down Vocabulary for Full ArXiv Corpus

In this notebook, we cut down on the number of unique tokens in the vocabulary. At the same time, we work to keep the vocabulary as full as (reasonably) possible to provide a realistic test of LDA on a large corpus of unknown documents.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
import copy
import editdistance
import matplotlib.pyplot as plt
import gensim
import os
import pandas as pd
import re
import sys
import time

sys.path.append('../../')
import tmx

## Dictionary and Corpus

In [ ]:
C_ROOT = '../../corpora/arxiv/arxiv-full-20M-spacy/'

### Dictionary

In [ ]:
dct = tmx.utils.unpickle(os.path.join(C_ROOT, 'arxiv-full-20M-spacy.mm.dictionary.cpickle'))

Most-used tokens:

In [ ]:
tokens = pd.DataFrame([
    {
        'token': dct[t],
        'docs': dct.dfs[t],
        'docs_frac': dct.dfs[t]/dct.num_docs
    } 
    for t in dct
])

In [ ]:
tokens = tokens.sort_values('docs', ascending=False)

In [ ]:
tokens.head(n=20)

### Corpus

In [ ]:
arxiv = gensim.corpora.MmCorpus(os.path.join(C_ROOT, 'arxiv-full-20M-spacy.mm'))

## Dictionary Thinning: Huge Corpus

In [ ]:
new_dct = copy.deepcopy(dct)

### Remove Least-Frequent Tokens

 - 74% of tokens appear in a single doc
 - 90% of tokens appear in fewer than 5 docs
 - 94% of tokens appear in fewer than 10 docs

In [ ]:
sum(tokens['docs'] < 2)/len(dct)

In [ ]:
sum(tokens['docs'] < 5)/len(dct)

In [ ]:
sum(tokens['docs'] < 10)/len(dct)

Remove the tokens that occur in just one document.

The goal is to remove obvious junk without being too opinionated. See, e.g., Lu, Kun, Xin Cai, Isola Ajiferuke, and Dietmar Wolfram. "Vocabulary size and its effect on topic representation." Information Processing & Management 53, no. 3 (2017): 653-665.

In [ ]:
new_dct.filter_extremes(no_below=2, no_above=1.00, keep_n=len(new_dct))
new_dct.compactify()

### Fix Punctuation

Eliminate or remap.

In [ ]:
RE_PUNCT = re.compile(r'''(^[^a-zA-Z0-9]+)|([^a-zA-Z0-9]+$)''')

In [ ]:
# Old token => New token, leaving not-found ones in place
remap_punct = {}
remapped_lst = []
for i, token in new_dct.iteritems():
    # Identity for non-edge-punct
    if not RE_PUNCT.search(token):
        remap_punct[token] = token
        continue
    new_token = RE_PUNCT.sub('', token)
    # REMOVE if the replacement isn't in the corpus
    if new_token not in new_dct.token2id:
        continue
    # Otherwise, remap to a similar token
    remapped_lst += [{token: new_token}]
    remap_punct[token] = new_token

In [ ]:
# Tokens to keep
new_dct.filter_tokens(good_ids=[new_dct.token2id[token] for token in remap_punct])
new_dct.compactify()

In [ ]:
len(new_dct)

### ABANDONED: Eliminate Similar Tokens

 - Find tokens that could be plural or a different tense
 - Find their partners

This seems too likely to cause new problems. Don't make any changes.

In [ ]:
for i, token in new_dct.iteritems():
    if not token.endswith('s'):
        continue
    # Find matching token
    new_token = token[:-1]
    print(i, token, new_token)
    if i > 300:
        break

### New Dictionary and Corpus

Two passes through the `VocabTransform`: once to convert old to old (punctuation) and once to convert old to new.

In [ ]:
new_dct.compactify()

In [ ]:
# Old to old. This will work because new_dct is a subset of dct.
map_o2o = {
    dct.token2id[old_token]: dct.token2id[new_token]
    for old_token, new_token in remap_punct.items()
}

vt = gensim.models.VocabTransform(map_o2o)
arxiv = vt[arxiv]

In [ ]:
# Old to new. The new dictionary only has the subset of tokens in it.
map_o2n = {
    dct.token2id[token]: new_id
    for new_id, token in new_dct.items()
}

vt = gensim.models.VocabTransform(map_o2n)
arxiv = vt[arxiv]

In [ ]:
new_dct.save('arxiv-full-trimmed-spacy.mm.dictionary.cpickle')

In [ ]:
gensim.corpora.MmCorpus.serialize('arxiv-full-trimmed-spacy.mm', arxiv)

## Dictionary Thinning: Top 100K Tokens

In [ ]:
new_dct2 = copy.deepcopy(dct)

In [ ]:
new_dct2.filter_extremes(no_below=0, no_above=1.0, keep_n=100000)
new_dct2.compactify()

### Fix Punctuation

Eliminate or remap.

In [ ]:
RE_PUNCT = re.compile(r'''(^[^a-zA-Z0-9]+)|([^a-zA-Z0-9]+$)''')

In [ ]:
# Old token => New token, leaving not-found ones in place
remap_punct2 = {}
remapped_lst2 = []
for i, token in new_dct2.iteritems():
    # Identity for non-edge-punct
    if not RE_PUNCT.search(token):
        remap_punct2[token] = token
        continue
    new_token = RE_PUNCT.sub('', token)
    # REMOVE if the replacement isn't in the corpus
    if new_token not in new_dct.token2id:
        continue
    # Otherwise, remap to a similar token
    remapped_lst2 += [{token: new_token}]
    remap_punct2[token] = new_token

In [ ]:
# Tokens to keep
new_dct2.filter_tokens(good_ids=[new_dct2.token2id[token] for token in remap_punct2])
new_dct2.compactify()

In [ ]:
len(new_dct2)

### New Dictionary and Corpus

Two passes through the `VocabTransform`: once to convert old to old (punctuation) and once to convert old to new.

In [ ]:
new_dct2.compactify()

In [ ]:
# Old to old. This will work because new_dct is a subset of dct.
map_o2o = {
    dct.token2id[old_token]: dct.token2id[new_token]
    for old_token, new_token in remap_punct2.items()
}

vt = gensim.models.VocabTransform(map_o2o)
arxiv = vt[arxiv]

In [ ]:
# Old to new. The new dictionary only has the subset of tokens in it.
map_o2n = {
    dct.token2id[token]: new_id
    for new_id, token in new_dct2.items()
}

vt = gensim.models.VocabTransform(map_o2n)
arxiv = vt[arxiv]

In [ ]:
new_dct2.save('arxiv-full-trimmed-100K-spacy.mm.dictionary.cpickle')

In [ ]:
gensim.corpora.MmCorpus.serialize('arxiv-full-trimmed-100K-spacy.mm', arxiv)